In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

function to find the best treshhold and alpha for feature selection, not very efficient

In [751]:
def treshholdfinder(alph,tresh):
    df=pd.read_csv('train.csv')
    df.head()
    df.dropna(axis=1, how='all')
    df = df.drop('Id', axis=1)
    numeric = df.select_dtypes(include=np.number).columns.tolist()
    numeric.remove('SalePrice')
    for i in numeric:
        df[i].fillna(int(df[i].mean()), inplace=True)
    scaler = MinMaxScaler()
    scaler_price = MinMaxScaler()
    df[numeric] = scaler.fit_transform(df[numeric])
    df['SalePrice']=scaler_price.fit_transform(df[['SalePrice']])
    unique_count = df.nunique()

# Filter the columns that have at most 4 unique values
    result_mean = unique_count[(unique_count <= 10)].index.tolist()
    df_backup=df.copy()
    for i in result_mean:
        Mean_encoded_subject = df.groupby([i])['SalePrice'].mean().to_dict()
  
        df[i] =  df[i].map(Mean_encoded_subject)
    df=df._get_numeric_data()
    columns=df.columns.values.tolist()
    for i in columns:
        df[i].fillna(int(df[i].median()), inplace=True)
    df.dropna()
    X_train, X_test, y_train, y_test= train_test_split(df.loc[:, df.columns != 'SalePrice'],df.iloc[:,-1],test_size=0.3)
    from sklearn.linear_model import Ridge
    from sklearn.feature_selection import SelectFromModel
    rr = Ridge(alpha=alph,random_state=0)
# Fit the model to the data
    rr.fit(X_train, y_train)
    
# Select features using a threshold
    sfm = SelectFromModel(rr, threshold=round(tresh,3))
    sfm.fit(X_train, y_train)

# Transform the feature matrix
    X_selected = sfm.transform(X_train)
    selected_feat= X_train.columns[(sfm.get_support())]
    selected_feat=selected_feat.tolist()
    len(selected_feat)
    df=pd.read_csv('train.csv')
    selected_feat.append('SalePrice')
    df=df[selected_feat]
    df.dropna(axis=1, how='all')
    df=df.dropna(thresh=len(df) - 400, axis=1)

    numeric = df.select_dtypes(include=np.number).columns.tolist()
    numeric.remove('SalePrice')
    for i in numeric:
        df[i].fillna(int(df[i].mean()), inplace=True)
    scaler = MinMaxScaler()
    scaler_price = MinMaxScaler()
    df[numeric] = scaler.fit_transform(df[numeric])
    df['SalePrice']=scaler_price.fit_transform(df[['SalePrice']])

    unique_count = df.nunique()

    result_mean = unique_count[(unique_count > 3) & (unique_count <= 10)].index.tolist()

    result_hot = unique_count[(unique_count <= 3)].index.tolist()

    df_backup=df.copy()
    for i in result_mean:
        Mean_encoded_subject = df.groupby([i])['SalePrice'].mean().to_dict()
  
        df[i] =  df[i].map(Mean_encoded_subject)
    for col in result_hot:
        col_encoded = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, col_encoded], axis=1)
        df.drop(col, axis=1, inplace=True)
    df=df._get_numeric_data()
    columns=df.columns.values.tolist()
    for i in columns:
        df[i].fillna(int(df[i].median()), inplace=True)
    df.dropna()
    df = df.assign(SalePrice=df.pop('SalePrice'))
    df_pr=df
    X_train, X_test, y_train, y_test= train_test_split(df_pr.loc[:, df_pr.columns != 'SalePrice'],df_pr.iloc[:,-1],test_size=0.3)
    from sklearn.ensemble import RandomForestRegressor
#rndm = RandomForestRegressor( n_estimators=1500,max_depth=20,random_state=0)
    rndm = RandomForestRegressor( random_state=0)
    rndm.fit(X_train, y_train)
    y_pred = rndm.predict(X_test)
# calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
# Print the metrics
    #print('Mean squared error (MSE):', mse)
    #print('Root mean squared error (RMSE):', rmse)
    #print('R-squared (R²) score:', r2)
    return r2




In [575]:
step_size=0.0005
i_best=0.001
tresh_best=0.7
alpha_best=0.1
for i in np.arange(0, 0.1+step_size, step_size):
    for j in np.arange(0, 0.1+step_size, step_size):
        treshi=treshholdfinder(i)
    #tresh_best=treshholdfinder(i)
        if tresh_best<treshi:
            print('changed'+str(tresh_best)+'to'+str(treshi)+'with i'+str(i))
            i_best=i
            tresh_best=treshi
        if i%.01==0:print(i,treshholdfinder(i),tresh_best)
print(tresh_best,i_best)

changed0.7to0.8926309367707275with i0.0
0.0 0.895435482600169 0.8926309367707275
changed0.8926309367707275to0.8956437880193956with i0.006500000000000001
0.01 0.8659891058151719 0.8956437880193956
0.02 0.8699602754938077 0.8956437880193956
changed0.8956437880193956to0.9043861814910813with i0.025500000000000002
changed0.9043861814910813to0.9132681612127653with i0.0315
0.04 0.8931467324919189 0.9132681612127653
0.08 0.8421173290804814 0.9132681612127653
0.9132681612127653 0.0315


In [806]:
df=pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [807]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [808]:
df = df.drop((missing_data[missing_data['Total'] > 1]).index,1)
df = df.drop(df.loc[df['Electrical'].isnull()].index)
df.isnull().sum().max() #just checking that there's no missing data missing...

/tmp/ipykernel_28350/1191676949.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop((missing_data[missing_data['Total'] > 1]).index,1)


0

In [809]:

df.sort_values(by = 'GrLivArea', ascending = False)[:2]
df = df.drop(df[df['Id'] == 1299].index)
df = df.drop(df[df['Id'] == 524].index)
df.dropna(axis=1, how='all')

In [811]:
#df=df.dropna(thresh=len(df) - 400, axis=1)
df = df.drop('Id', axis=1)

In [812]:
numeric = df.select_dtypes(include=np.number).columns.tolist()
numeric.remove('SalePrice')

In [813]:
#columns=df.columns.values.tolist()
for i in numeric:
    df[i].fillna(int(df[i].mean()), inplace=True)

In [814]:
scaler = MinMaxScaler()
scaler_price = MinMaxScaler()
#df[numeric] = scaler.fit_transform(df[numeric])
#df['SalePrice']=scaler_price.fit_transform(df[['SalePrice']])

In [815]:
unique_count = df.nunique()

# Filter the columns that have at most 4 unique values
result_mean = unique_count[(unique_count <= 10)].index.tolist()


In [816]:
df_backup=df.copy()
for i in result_mean:
    Mean_encoded_subject = df.groupby([i])['SalePrice'].mean().to_dict()
  
    df[i] =  df[i].map(Mean_encoded_subject)

In [818]:
df=df._get_numeric_data()
columns=df.columns.values.tolist()
for i in columns:
    df[i].fillna(int(df[i].median()), inplace=True)
df.dropna()

In [820]:
X_train=df.loc[:, df.columns != 'SalePrice']
y_train=df.iloc[:,-1]

In [821]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectFromModel
rr = Ridge(alpha=0.004,random_state=0)
# Fit the model to the data
rr.fit(X_train, y_train)

# Select features using a threshold
sfm = SelectFromModel(rr, threshold=0.0315)
sfm.fit(X_train, y_train)

# Transform the feature matrix
X_selected = sfm.transform(X_train)

In [822]:
selected_feat= X_train.columns[(sfm.get_support())]
selected_feat=selected_feat.tolist()
len(selected_feat)

52

In [841]:
df=pd.read_csv('train.csv')

selected_feat.append('SalePrice')
df=df[selected_feat]
df.dropna(axis=1, how='all')
df=df.dropna(thresh=len(df) - 400, axis=1)

numeric = df.select_dtypes(include=np.number).columns.tolist()
print(numeric)
numeric.remove('SalePrice')
for i in numeric:
    df[i].fillna(float(df[i].mean()), inplace=True)
scaler = MinMaxScaler()
scaler_price = MinMaxScaler(feature_range=(0.1, 1))
df[numeric] = scaler.fit_transform(df[numeric])
df['SalePrice']=scaler_price.fit_transform(df[['SalePrice']])

unique_count = df.nunique()

result_mean = unique_count[(unique_count  > 2) & (unique_count <= 10)].index.tolist()

#result_hot = unique_count[(unique_count <= 5)& (unique_count > 2)].index.tolist()

df_backup=df.copy()
for i in result_mean:
    Mean_encoded_subject = df.groupby([i])['SalePrice'].mean().to_dict()
  
    df[i] =  df[i].map(Mean_encoded_subject)
'''for col in result_hot:
    col_encoded = pd.get_dummies(df[col], prefix=col)
    df = pd.concat([df, col_encoded], axis=1)
    df.drop(col, axis=1, inplace=True)'''
df=df._get_numeric_data()
columns=df.columns.values.tolist()
for i in columns:
    df[i].fillna(int(df[i].median()), inplace=True)
df.dropna()
df = df.assign(SalePrice=df.pop('SalePrice'))
df['SalePrice']=np.log(df['SalePrice'])
df_pr=df

['MSSubClass', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'SalePrice']


In [843]:
X_train, X_test, y_train, y_test= train_test_split(df_pr.loc[:, df_pr.columns != 'SalePrice'],df_pr.loc[:, df_pr.columns == 'SalePrice'],test_size=0.3,random_state=3)

In [844]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#rndm = RandomForestRegressor(random_state=0)
#rndm.fit(X_train, y_train)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
 #calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

Mean squared error (MSE): 0.0188274020577239
Root mean squared error (RMSE): 0.13721298064587
R-squared (R²) score: 0.7909552478974471


In [845]:
from sklearn import svm
regr = svm.SVR(kernel='rbf',gamma='scale',C=1.3)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

Mean squared error (MSE): 0.008693360328546293
Root mean squared error (RMSE): 0.09323819136247921
R-squared (R²) score: 0.9034757238812148


/home/almighty/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [846]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

Mean squared error (MSE): 0.019135747150693442
Root mean squared error (RMSE): 0.13833201780749618
R-squared (R²) score: 0.7875316250670518


In [847]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

Mean squared error (MSE): 0.03461494885083311
Root mean squared error (RMSE): 0.1860509307980831
R-squared (R²) score: 0.6156626719193914


/home/almighty/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [848]:
from sklearn.linear_model import HuberRegressor
huber = HuberRegressor()
huber.fit(X_train, y_train)
y_pred = huber.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

/home/almighty/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean squared error (MSE): 0.023572600361751073
Root mean squared error (RMSE): 0.15353371083169673
R-squared (R²) score: 0.7382682759986414


/home/almighty/conda/lib/python3.10/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [849]:
from sklearn.ensemble import RandomForestRegressor
#rndm = RandomForestRegressor( n_estimators=1500,max_depth=20,random_state=0)
rndm = RandomForestRegressor( random_state=0)
rndm.fit(X_train, y_train)
y_pred = rndm.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

/tmp/ipykernel_28350/1759980802.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rndm.fit(X_train, y_train)


Mean squared error (MSE): 0.012617144944729305
Root mean squared error (RMSE): 0.11232606529532362
R-squared (R²) score: 0.8599090873437385


In [850]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(max_depth=5)

tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
# calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Print the metrics
print('Mean squared error (MSE):', mse)
print('Root mean squared error (RMSE):', rmse)
print('R-squared (R²) score:', r2)

Mean squared error (MSE): 0.02318295887039695
Root mean squared error (RMSE): 0.15225951159253384
R-squared (R²) score: 0.7425945504744965


In [851]:
df_tst=pd.read_csv('test.csv')
df_bc=df_tst.copy()
selected_feat.remove('SalePrice')
df_tst=df_tst[selected_feat]

In [852]:
df_tst[numeric] = scaler.fit_transform(df_tst[numeric])

In [853]:
for i in result_mean:
    Mean_encoded_subject = df_backup.groupby([i])['SalePrice'].mean().to_dict()
  
    df_tst[i] =  df_tst[i].map(Mean_encoded_subject)
'''for col in result_hot:
    col_encoded = pd.get_dummies(df_tst[col], prefix=col)
    df_tst = pd.concat([df_tst, col_encoded], axis=1)
    df_tst.drop(col, axis=1, inplace=True)'''
#df_tst = df_tst.drop('Id', axis=1)
numeric = df_tst.select_dtypes(include=np.number).columns.tolist()
for i in numeric:
    df_tst[i].fillna(float(df_tst[i].mean()), inplace=True)

In [854]:
df_tst=df_tst._get_numeric_data()
columns=df_tst.columns.values.tolist()
for i in columns:
    df_tst[i].fillna(int(df_tst[i].median()), inplace=True)

In [863]:
df_bc['SalePrice']=regr.predict(df_tst)

In [865]:
df_bc['SalePrice'] = np.exp(df_bc[['SalePrice']])
df_bc['SalePrice'] = scaler_price.inverse_transform(df_bc[['SalePrice']])

In [805]:
df_out=df_bc.iloc[:, [0] + [-1]]
df_out.to_csv('output.csv', index=False)